# Project Setup

This notebook will guide you through setting up the project environment for using CrewAI. We will:

1. Install the required Python modules.
2. Set up a virtual environment.
3. Verify the installation.

In [ ]:
# Step 1: Create and activate a virtual environment
%python3 -m venv venv
%source venv/bin/activate

# Step 2: Install required Python modules
%pip install -r requirements.txt

# Step 3: Verify installation
%pip list

# Load Required Python Modules and Libraries

In [1]:
# Step 4: Set OpenAI API Key from .env file
import os
from dotenv import load_dotenv
from IPython.display import Markdown
from crewai import LLM, Agent, Task, Crew

# Load Environment Variables and Configure LLM

This block loads environment variables from the `.env` file, including the OpenAI API Key, which is required to authenticate with OpenAI's services. It then configures the `LLM` object to use OpenAI's GPT-4 model. Alternatively, you can uncomment the provided code to configure the `LLM` object to use Ollama with a local model, provided Ollama is installed and running.

In [ ]:
# Load environment variables from .env file
load_dotenv()

# Uncomment the code block below to use OpenAI with your API Key
api_key = os.getenv('OPENAI_API_KEY')
if not api_key:
    raise ValueError("OPENAI_API_KEY is not set in the .env file")

# Configure LLM

Configures the `LLM` object to use OpenAI's GPT-4 model. 

Alternatively, you can uncomment the provided code to configure the `LLM` object to use Ollama with a local model, provided Ollama is installed and running.

In [ ]:
llm = LLM(
    model="gpt-4o",  # Specify the OpenAI model you want to use
    api_key=api_key
)

# Uncomment the code block below to use Ollama with your local model
# Make sure to have Ollama installed and running
# 
# llm = LLM(
#     model="ollama/llama3.2:1b",
#     base_url="http://localhost:11434"
# )

# Single Agent Single Tool Example

## Define Agents

This block defines multiple agents with specific roles, goals, and backstories. Each agent is configured to use the LLM defined earlier.

In [2]:
senior_technical_writer = Agent(
    role="Senior Technical Writer",
    
    goal="""Craft clear, engaging, and well-structured
            technical content based on research findings""",
    
    backstory="""You are an experienced technical writer
                with expertise in simplifying complex
                concepts, structuring content for readability,
                and ensuring accuracy in documentation.""",
                
    llm=llm,
                
    verbose=True
)

research_analyst = Agent(
    role="Senior Research Analyst",
    goal="""Find, analyze, and summarize information 
            from various sources to support technical 
            and business-related inquiries.""",
    backstory="""You are a skilled research analyst with expertise 
                in gathering accurate data, identifying key trends, 
                and presenting insights in a structured manner.""",
    llm=llm,
    verbose=True
)

code_reviewer = Agent(
    role="Senior Code Reviewer",
    goal="""Review code for bugs, inefficiencies, and 
            security vulnerabilities while ensuring adherence 
            to best coding practices.""",
    backstory="""You are a seasoned software engineer with years of 
                experience in writing, reviewing, and optimizing 
                production-level code in multiple programming languages.""",
    llm=llm,
    verbose=True
)

legal_reviewer = Agent(
    role="Legal Document Expert Reviewer",
    goal="""Review contracts and legal documents to 
            ensure compliance with applicable laws and 
            highlight potential risks.""",
    backstory="""You are a legal expert with deep knowledge 
                of contract law, regulatory frameworks, 
                and risk mitigation strategies.""",
    llm=llm,
    verbose=True
)


## Define a Writing Task

This block defines a writing task for the Senior Technical Writer agent. The task includes a description, the agent responsible, and the expected output.

In [3]:

writing_task = Task(
    description="""Write a well-structured, engaging,
                   and technically accurate article
                   on {topic}.""",
    
    agent=senior_technical_writer, 
    
    
    expected_output="""A polished, detailed, and easy-to-read
                       article on the given topic.""",
)


## Create a Crew and Execute the Task

This block creates a crew to manage the task and agents. It then kicks off the crew with the specified input and displays the response.

In [ ]:
crew = Crew(
    agents=[senior_technical_writer],
    tasks=[writing_task],
    verbose=True
)

response = crew.kickoff(inputs={"topic":"AI Agents"})

# Display the response in Markdown format
# Markdown(response.raw)

## Save the Response to a File

This block saves the raw response from the crew execution to a Markdown file for further use or documentation.

In [11]:
f = open("build/output.md", "w")
f.write(response.raw)
f.close()

## Define a file reader tool

In [12]:
from crewai_tools import FileReadTool

file_read_tool = FileReadTool()

## Define the Agent For Reading And Summarizing Files

In [23]:
from crewai import Agent

summarizer_agent = Agent(
    role="Senior Document Summarizer",
    # goal="Summarize articles in exactly 20 words.",
    # backstory="You are an expert at creating concise summaries.",
    # llm=llm,
    goal="Extract and summarize key insights from provided files in 20 words or less.",
    backstory="""You are an expert in document analysis, skilled at extracting 
                 key details, summarizing content, and identifying critical 
                 insights from structured and unstructured text.""",
    llm=llm,
    tools=[file_read_tool],
    verbose=True
)

## Create Reading and Analyzing File Task

In [24]:
from crewai import Task

summarizer_task = Task(
    description=(
        "Use the FileReadTool to read the contents of {file_path}"
        "and provide a summary in 20 words or less. "
        "Ensure the summary captures the key insights "
        "and main points from the document."
    ),
    agent=summarizer_agent,
    tools=[file_read_tool],
    expected_output="A concise 20-word summary of the key points from the file.",
)

## Assemble A File-Processing Crew Workflow

In [ ]:
from crewai import Crew

summarizer_crew = Crew(
    agents=[summarizer_agent],
    tasks=[summarizer_task],
    verbose=True
)

# Run the crew with the file input
result = summarizer_crew.kickoff(inputs={"file_path": "build/output.md"})

In [26]:
# Display the response in Markdown format
from IPython.display import Markdown
Markdown(result.raw)

AI agents autonomously learn, adapt, and interact, revolutionizing industries like healthcare and finance but pose complexity and ethical challenges.

# Building multi-agent systems

## Define our Serper Dev tool

The Serper Dev Tool is a utility designed to interact with search engines and retrieve relevant information programmatically. It is particularly useful for tasks that require real-time data or insights from the web, such as:

1. Conducting research on specific topics.
2. Gathering up-to-date information for decision-making.
3. Enhancing the capabilities of agents by providing them with access to external data sources.

### How It Can Be Used
- **Integration with Agents**: The tool can be integrated into agents to enable them to fetch and process web-based information dynamically.
- **Custom Queries**: Developers can define specific queries to retrieve targeted information, making it adaptable to various use cases.
- **Real-Time Insights**: By leveraging the tool, agents can provide real-time insights and context, improving the quality and relevance of their outputs.

This tool is ideal for scenarios where static data is insufficient, and dynamic, real-time information is required to achieve the desired outcomes.

In [2]:
from crewai_tools import SerperDevTool

serper_dev_tool = SerperDevTool()

/Users/toddwalters/Development/repos/personal/ai-agents-crash-course/venv/lib/python3.11/site-packages/pydantic/_internal/_generate_schema.py:623: UserWarning: <built-in function callable> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(


## Define the Internet Researcher Agent and Task

This block defines an `Internet Researcher` agent and a corresponding research task. 
- The agent is equipped with the `SerperDevTool` to perform web-based research. 
- The agent's role is to find the most relevant and recent information about a given topic, leveraging its expertise in navigating the internet and gathering reliable data. 
- The task specifies the use of the `SerperDevTool` to extract key insights from multiple sources and produce a detailed research report with references.

In [3]:
from crewai import Agent, Task

research_agent = Agent(
    role="Internet Researcher",
    goal="Find the most relevant and recent information about a given topic.",
    backstory="""You are a skilled researcher, adept at navigating the internet 
                 and gathering high-quality, reliable information.""",
    tools=[serper_dev_tool],
    verbose=True
)

research_task = Task(
    description="""Use the SerperDevTool to search for the 
                   most relevant and recent data about {topic}."""
                "Extract the key insights from multiple sources.",
    agent=research_agent,
    tools=[serper_dev_tool],
    expected_output="A detailed research report with key insights and source references."
)

## Define The Summarization Agent

This agent is responsible for condensing the research into a concise and structured summary. The Summarization Agent ensures that the research findings are structured, easy to read, and clear.

In [4]:
summarizer_agent = Agent(
    role="Content Summarizer",
    goal="Condense the key insights from research into a short and informative summary.",
    backstory="""You are an expert in distilling complex information into concise, 
                 easy-to-read summaries.""",
    verbose=True
)

summarization_task = Task(
    description="Summarize the research report into a concise and informative paragraph. "
                "Ensure clarity, coherence, and completeness.",
    agent=summarizer_agent,
    expected_output="A well-structured summary with the most important insights."
)

## Define The Fact-Checking Agent

The Fact-Checking Agent will cross-check all summarized information with credible sources:

- The Fact-Checking Agent is responsible for validating the summarized information.
- The Serper Dev Tool is used again to cross-check facts with external sources.

In [5]:
fact_checker_agent = Agent(
    role="Fact-Checking Specialist",
    goal="Verify the accuracy of information and remove any misleading or false claims.",
    backstory="""You are an investigative journalist with a knack for validating facts, 
                 ensuring that only accurate information is published.""",
    tools=[serper_dev_tool],
    verbose=True
)

fact_checking_task = Task(
    description="Verify the summarized information for accuracy using the SerperDevTool. "
                "Cross-check facts with reliable sources and correct any errors.",
    agent=fact_checker_agent,
    tools=[serper_dev_tool],
    expected_output="A fact-checked, verified summary of the research topic."
)

## Create Multi-Agent Crew Workflow

- All three agents are grouped into a Crew, each assigned their specific task.
- Tasks are executed sequentially in a structured workflow:  Research → Summarization → Fact-Checking.
- The topic is dynamically provided at runtime, making the workflow flexible for any research topic.

In [6]:
from crewai import Crew, Process

research_crew = Crew(
    agents=[research_agent, summarizer_agent, fact_checker_agent],
    tasks=[research_task, summarization_task, fact_checking_task],
    process=Process.sequential,
    verbose=True
)


## Kickoff The Multi-Agent Crew Workflow

In [8]:

result = research_crew.kickoff(inputs={"topic": "The impact of GLP-1 receptor on weight-loss markets"})
print("\nFinal Verified Summary:\n", result)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 7d20bab7-dc61-4ad7-a5b2-8272a0a75e33                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 760c4a79-a15a-4c56-8c00-1b33c3cf599d
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 760c4a79-a15a-4c56-8c00-1b33c3cf599d
       Status: Executing Task...
    └── 🤖 Agent: Internet Researcher
            Status: In Progress

# Agent: Internet Researcher
## Task: Use the SerperDevTool to search for the 
                   most relevant and recent data about The impact of GLP-1 receptor on weight-loss markets.Extract the key insights from multiple sources.


🤖 Agent: Internet Researcher
    Status: In Progress

...

🚀 Crew: crew
├── 📋 Task: 760c4a79-a15a-4c56-8c00-1b33c3cf599d
│      Assigned to: Internet Researcher
│      Status: ✅ Completed
│   └── 🤖 Agent: Internet Researcher
│           Status: ✅ Completed
├── 📋 Task: a6e10545-b598-4732-a9c8-1129703a36cc
│      Assigned to: Content Summarizer
│      Status: ✅ Completed
│   └── 🤖 Agent: Content Summarizer
│           Status: ✅ Completed
└── 📋 Task: 88fc4d1d-ba82-4cc3-9667-344aa8682729
       Status: Executing Task...
    └── 🤖 Agent: Fact-Checking Specialist
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: 760c4a79-a15a-4c56-8c00-1b33c3cf599d
│      Assigned to: Internet Researcher
│      Status: ✅ Completed
│   └── 🤖 Agent: Internet Researcher
│           Status: ✅ Completed
├── 📋 Task: a6e10545-b598-4732-a9c8-1129703a36cc
│      Assigned to: Content Summarizer
│      Status: ✅ Completed
│   └── 🤖 Agent: Content Summarizer
│           Status: ✅ Completed
└── 📋 Task: 88fc4d1d-ba82-4cc3-9667-344aa8682729
       Assigned to: Fact-Checking Specialist
       Status: ✅ Completed
    └── 🤖 Agent: Fact-Checking Specialist
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 88fc4d1d-ba82-4cc3-9667-344aa8682729                                                                     │
│  Agent: Fact-Checking Specialist                                                                                │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 7d20bab7-dc61-4ad7-a5b2-8272a0a75e33                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯


Final Verified Summary:
 The impact of GLP-1 receptor agonists on weight-loss markets is extensive, with the market projected to grow dramatically from approximately $36.8 billion in 2023 to between $55.7 billion and $138 billion by 2031, featuring compound annual growth rates (CAGR) between approximately 10.3% and 19.9%. These medications effectively suppress appetite and enhance feelings of fullness, which are crucial in weight management. They have significant economic implications, leading to reduced grocery spending among users and potentially transforming the food industry. Future trends indicate continued innovation in GLP-1 medications, with new formulations on the horizon as the landscape for obesity and diabetes treatments evolves. Consumer adoption of GLP-1 drugs is expected to be considerable, emphasizing their role in reshaping both personal and broader economic behavior.
